## 前言
教程见官网:(十分详细)
https://www.tensorflow.org/guide/datasets#dataset_structure  
借助 tf.data API，您可以根据简单的可重用片段构建复杂的输入管道。例如，图片模型的管道可能会汇聚分布式文件系统中的文件中的数据、对每个图片应用随机扰动，并将随机选择的图片合并成用于训练的批次。文本模型的管道可能包括从原始文本数据中提取符号、根据对照表将其转换为嵌入标识符，以及将不同长度的序列组合成批次数据。使用 tf.data API 可以轻松处理大量数据、不同的数据格式以及复杂的转换。

tf.data API 在 TensorFlow 中引入了两个新的抽象类：

* `tf.data.Dataset` 表示一系列元素，其中每个元素包含一个或多个 Tensor 对象。例如，在图像管道中，元素可能是单个训练样本，具有一对表示图像数据和标签的张量。可以通过两种不同的方式来创建数据集：  
创建来源（例如 `Dataset.from_tensor_slices()`），以通过一个或多个 `tf.Tensor` 对象构建数据集。  
应用转换（例如 `Dataset.batch()`），以通过一个或多个 `tf.data.Dataset` 对象构建数据集。

* `tf.data.Iterator` 提供了从数据集中提取元素的主要方法。`Iterator.get_next()` 返回的操作会在执行时生成 Dataset 的下一个元素，并且此操作通常充当输入管道代码和模型之间的接口。最简单的迭代器是“单次迭代器”，它与特定的 Dataset 相关联，并对其进行一次迭代。要实现更复杂的用途，您可以通过 Iterator.initializer 操作使用不同的数据集重新初始化和参数化迭代器，这样一来，您就可以在同一个程序中对训练和验证数据进行多次迭代（举例而言）。

总之,`tf.data.Dataset` 来处理数据集会使代码变得简单，也是目前TensorFlow主推的一种数据集处理方式.  
有了Dataset对象之后，就可以在其上直接做**乱序(shuffle)、元素变换(map), 按批次组合(batch), 重复(repeat)、迭代取值(iterate)** 等操作.  
Dataset对象可以由不同数据源转化得到.  
在tf.data.Dataset接口中,有三种方法可将内存中的数据转化成Dataset对象:  
* `tf.data.Dataset.from_tensor`: 内存对象中只有一个元素.  
* **`tf.data.Dataset.from_tensor_slices`**: 内存对象可以是列表,元组,字典,Numpy数组等类型. 另外,该方法也支持Tensorflow中的张量类型.  
* `tf.data.Dataset.from_generator`: 根据生成器对象生产Dataset对象. 

In [16]:
import tensorflow as tf
Dataset = tf.data.Dataset

## `Dataset.range(*args)` 
与python语法中的range()基本等价.

Signature: `Dataset.range(*args)`  
Docstring:  
Creates a `Dataset` of a step-separated range of values.

For example:

```python
Dataset.range(5) == [0, 1, 2, 3, 4]
Dataset.range(2, 5) == [2, 3, 4]
Dataset.range(1, 5, 2) == [1, 3]
Dataset.range(1, 5, -2) == []
Dataset.range(5, 1) == []
Dataset.range(5, 1, -2) == [5, 3]
```

In [19]:
Dataset.range(5)

<RangeDataset shapes: (), types: tf.int64>

In [18]:
for i in Dataset.range(5):
    i

<tf.Tensor: id=128, shape=(), dtype=int64, numpy=0>

<tf.Tensor: id=130, shape=(), dtype=int64, numpy=1>

<tf.Tensor: id=132, shape=(), dtype=int64, numpy=2>

<tf.Tensor: id=134, shape=(), dtype=int64, numpy=3>

<tf.Tensor: id=136, shape=(), dtype=int64, numpy=4>

## `Dataset.zip(datasets)` 
与python语法中的zip基本等价:

```python
a = Dataset.range(1, 4)  # ==> [ 1, 2, 3 ]  
b = Dataset.range(4, 7)  # ==> [ 4, 5, 6 ]  
c = Dataset.range(7, 13).batch(2)  # ==> [ [7, 8], [9, 10], [11, 12] ]  
d = Dataset.range(13, 15)  # ==> [ 13, 14 ]  

# The nested structure of the `datasets` argument determines the
# structure of elements in the resulting dataset.
Dataset.zip((a, b))  # ==> [ (1, 4), (2, 5), (3, 6) ]
Dataset.zip((b, a))  # ==> [ (4, 1), (5, 2), (6, 3) ]

# The `datasets` argument may contain an arbitrary number of
# datasets.
Dataset.zip((a, b, c))  # ==> [ (1, 4, [7, 8]),
                        #       (2, 5, [9, 10]),
                        #       (3, 6, [11, 12]) ]

# The number of elements in the resulting dataset is the same as
# the size of the smallest dataset in `datasets`.
Dataset.zip((a, d))  # ==> [ (1, 13), (2, 14) ]
```

Args:
  datasets: A nested structure of datasets.

In [20]:
a = Dataset.range(1, 4)  # ==> [ 1, 2, 3 ]  
b = Dataset.range(4, 7)  # ==> [ 4, 5, 6 ]  
c = Dataset.range(7, 13).batch(2)  # ==> [ [7, 8], [9, 10], [11, 12] ]  
d = Dataset.range(13, 15)  # ==> [ 13, 14 ]  
Dataset.zip((a, b))

<ZipDataset shapes: ((), ()), types: (tf.int64, tf.int64)>

## `Dataset.from_tensor_slices(tensors)`  


Creates a `Dataset` whose elements are slices of the given tensors.

Args:
  tensors: A nested structure of tensors, each having the same size in the 0th dimension.  
Args: 张量:张量的嵌套结构，每个张量在第0维度上具有相同的大小。

In [26]:
a = Dataset.from_tensor_slices([1,2,3,4,5])
a
for i in a:
    i

<TensorSliceDataset shapes: (), types: tf.int32>

<tf.Tensor: id=172, shape=(), dtype=int32, numpy=1>

<tf.Tensor: id=174, shape=(), dtype=int32, numpy=2>

<tf.Tensor: id=176, shape=(), dtype=int32, numpy=3>

<tf.Tensor: id=178, shape=(), dtype=int32, numpy=4>

<tf.Tensor: id=180, shape=(), dtype=int32, numpy=5>

In [29]:
b = Dataset.from_tensor_slices(np.random.randn(2,5))
b

for i in b:
    i

<TensorSliceDataset shapes: (5,), types: tf.float64>

<tf.Tensor: id=193, shape=(5,), dtype=float64, numpy=array([ 0.0263448 , -0.91094336,  0.47878163,  0.01739943,  0.29555386])>

<tf.Tensor: id=195, shape=(5,), dtype=float64, numpy=array([ 0.25180477,  1.92013687, -1.20406819,  0.2064275 , -1.52603222])>

## `Dataset.repeat(self, count=None)` 
Repeats this dataset `count` times.

NOTE: If this dataset is a function of global state (e.g. a random number
generator), then different repetitions may produce different elements.

Args:  
  count: (Optional.) A `tf.int64` scalar `tf.Tensor`, representing the
    number of times the dataset should be repeated.   
    The default behavior (if`count` is `None` or `-1`) is for the dataset be repeated indefinitely.  
    默认行为（如果 `count`是`None`或`-1`）数据集是无限重复的。

In [36]:
a = Dataset.range(3)
for i in Dataset.repeat(a, 2):
    i

<tf.Tensor: id=6265, shape=(), dtype=int64, numpy=0>

<tf.Tensor: id=6267, shape=(), dtype=int64, numpy=1>

<tf.Tensor: id=6269, shape=(), dtype=int64, numpy=2>

<tf.Tensor: id=6271, shape=(), dtype=int64, numpy=0>

<tf.Tensor: id=6273, shape=(), dtype=int64, numpy=1>

<tf.Tensor: id=6275, shape=(), dtype=int64, numpy=2>

In [37]:
for i in a.repeat(2):
    i

<tf.Tensor: id=6284, shape=(), dtype=int64, numpy=0>

<tf.Tensor: id=6286, shape=(), dtype=int64, numpy=1>

<tf.Tensor: id=6288, shape=(), dtype=int64, numpy=2>

<tf.Tensor: id=6290, shape=(), dtype=int64, numpy=0>

<tf.Tensor: id=6292, shape=(), dtype=int64, numpy=1>

<tf.Tensor: id=6294, shape=(), dtype=int64, numpy=2>

## `Dataset.concatenate(self, dataset)` 

Creates a `Dataset` by concatenating given dataset with this dataset.

```python
a = Dataset.range(1, 4)  # ==> [ 1, 2, 3 ]
b = Dataset.range(4, 8)  # ==> [ 4, 5, 6, 7 ]

# Input dataset and dataset to be concatenated should have same
# nested structures and output types.
# c = Dataset.range(8, 14).batch(2)  # ==> [ [8, 9], [10, 11], [12, 13] ]
# d = Dataset.from_tensor_slices([14.0, 15.0, 16.0])
# a.concatenate(c) and a.concatenate(d) would result in error.

a.concatenate(b)  # ==> [ 1, 2, 3, 4, 5, 6, 7 ]
```

Args:
  dataset: `Dataset` to be concatenated.

Returns:
  Dataset: A `Dataset`.

## `Dataset.shuffle`
```python
Dataset.shuffle(
    self,
    buffer_size,
    seed=None,
    reshuffle_each_iteration=None,
)
```
Docstring:
Randomly shuffles the elements of this dataset.

This dataset fills a buffer with `buffer_size` elements, then randomly
samples elements from this buffer, replacing the selected elements with new
elements. For perfect shuffling, a buffer size greater than or equal to the
full size of the dataset is required.  
此数据集使用`buffer_size`元素填充缓冲区，然后随机填充 对此缓冲区中的元素进行采样，用new替换所选元素 元素。为了完美的改组，缓冲区大小大于或等于 需要完整大小的数据集。

For instance, if your dataset contains 10,000 elements but `buffer_size` is
set to 1,000, then `shuffle` will initially select a random element from
only the first 1,000 elements in the buffer. Once an element is selected,
its space in the buffer is replaced by the next (i.e. 1,001-st) element,
maintaining the 1,000 element buffer.  
例如，如果您的数据集包含10,000个元素，但“buffer_size”是 设置为1,000，然后`shuffle`将首先从中选择一个随机元素 只有缓冲区中的前1,000个元素。选择元素后， 它在缓冲区中的空间被下一个（即1,001-st）元素替换， 维护1,000个元素缓冲区。

Args:  
  buffer_size: A `tf.int64` scalar `tf.Tensor`, representing the number of
    elements from this dataset from which the new dataset will sample.   
    代表新数据集将从原始数据集中进行采样的数量。

    
  seed: (Optional.) A `tf.int64` scalar `tf.Tensor`, representing the random
    seed that will be used to create the distribution. See
    `tf.compat.v1.set_random_seed` for behavior.   
    随机种子.
    
  reshuffle_each_iteration: (Optional.) A boolean, which if true indicates
    that the dataset should be pseudorandomly reshuffled each time it is
    iterated over. (Defaults to `True`.)  
    是否每一次迭代都重新洗牌.

In [52]:
a = Dataset.range(1, 7)

for i in a.shuffle(buffer_size = 1):
    i
print(55*'_')
for i in a.shuffle(buffer_size = 2):
    i
print(55*'_')
for i in a.shuffle(buffer_size = 3):
    i

<tf.Tensor: id=6874, shape=(), dtype=int64, numpy=1>

<tf.Tensor: id=6876, shape=(), dtype=int64, numpy=2>

<tf.Tensor: id=6878, shape=(), dtype=int64, numpy=3>

<tf.Tensor: id=6880, shape=(), dtype=int64, numpy=4>

<tf.Tensor: id=6882, shape=(), dtype=int64, numpy=5>

...

<tf.Tensor: id=6918, shape=(), dtype=int64, numpy=4>

<tf.Tensor: id=6920, shape=(), dtype=int64, numpy=1>

<tf.Tensor: id=6922, shape=(), dtype=int64, numpy=6>

<tf.Tensor: id=6924, shape=(), dtype=int64, numpy=5>

<tf.Tensor: id=6926, shape=(), dtype=int64, numpy=2>

## `Dataset.batch`
`Dataset.batch(self, batch_size, drop_remainder=False)`

Combines consecutive elements of this dataset into batches.
将此数据集的连续元素组合成批次。 

The tensors in the resulting element will have an additional outer
dimension, which will be `batch_size` (or `N % batch_size` for the last
element if `batch_size` does not divide the number of input elements `N`
evenly and `drop_remainder` is `False`). If your program depends on the
batches having the same outer dimension, you should set the `drop_remainder`
argument to `True` to prevent the smaller batch from being produced.  

结果元素中的张量将具有额外维度`batch_size`（或者是最后的`N％batch_size` 如果`batch_size`没有划分输入元素的数量`N`，则为element 均匀地和`drop_remainder`是`False`）。如果你的程序依赖于具有相同额外维度(i.e batch size)，您应该设置`drop_remainder` 参数为“True”以防止产生较小的批次。

Args:  
  batch_size: 批次大小
  drop_remainder: (Optional.)代表是否应该删除最后一批，如果它少于`batch_size`个元素;  
  默认行为是不要删除较小的批量

In [60]:
a = Dataset.range(1,99)
for i in a.batch(20):
    i

<tf.Tensor: id=7087, shape=(20,), dtype=int64, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])>

<tf.Tensor: id=7089, shape=(20,), dtype=int64, numpy=
array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40])>

<tf.Tensor: id=7091, shape=(20,), dtype=int64, numpy=
array([41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60])>

<tf.Tensor: id=7093, shape=(20,), dtype=int64, numpy=
array([61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80])>

<tf.Tensor: id=7095, shape=(18,), dtype=int64, numpy=
array([81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
       98])>

In [61]:
for i in a.batch(20, drop_remainder=True):
    i

<tf.Tensor: id=7107, shape=(20,), dtype=int64, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])>

<tf.Tensor: id=7109, shape=(20,), dtype=int64, numpy=
array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40])>

<tf.Tensor: id=7111, shape=(20,), dtype=int64, numpy=
array([41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60])>

<tf.Tensor: id=7113, shape=(20,), dtype=int64, numpy=
array([61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80])>

## Dataset.prefetch(buffer_size)

Creates a `Dataset` that prefetches elements from this dataset.  
创建一个“数据集”，用于预取此数据集中的元素。  

Note: Like other `Dataset` methods, prefetch operates on the
elements of the input dataset. It has no concept of examples vs. batches.  
`examples.prefetch(2)` will prefetch two elements (2 examples),
while `examples.batch(20).prefetch(2)` will prefetch 2 elements
(2 batches, of 20 examples each).   
 注意：与其他`Dataset`方法一样，prefetch是操作在 输入数据集的元素。它没有例子与批量的概念。  
 `examples.prefetch（2）`将预取两个元素（2个例子）， 而`examples.batch（20）.prefetch（2）`将预取2个元素 （2批，每批20个例子）。


In [67]:
a = Dataset.range(100)
for i in a.batch(20).prefetch(3): # 跟预想的不一致
    i

<tf.Tensor: id=7253, shape=(20,), dtype=int64, numpy=
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])>

<tf.Tensor: id=7255, shape=(20,), dtype=int64, numpy=
array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39])>

<tf.Tensor: id=7257, shape=(20,), dtype=int64, numpy=
array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59])>

<tf.Tensor: id=7259, shape=(20,), dtype=int64, numpy=
array([60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
       77, 78, 79])>

<tf.Tensor: id=7261, shape=(20,), dtype=int64, numpy=
array([80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96,
       97, 98, 99])>